In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#@title Extraction Algorithm

def extract_code_blocks(file_path):
  with open(file_path, 'r') as file:
    lines = file.readlines()

  user_code_blocks = []
  assistant_code_blocks = []
  current_block = []
  current_speaker = None

  for line in lines:
    if line.startswith(">> USER:"):
      if current_speaker == "assistant":
       assistant_code_blocks.append("".join(current_block).strip())
      current_block = []
      current_speaker = "user"
      current_block.append(line[len(">> USER:"):])
    elif line.startswith(">> ASSISTANT:"):
      if current_speaker == "user":
        user_code_blocks.append("".join(current_block).strip())
      current_block = []
      current_speaker = "assistant"
      current_block.append(line[len(">> ASSISTANT:"):])
    else:
      if current_speaker:
        current_block.append(line)

  if current_speaker == "user":
    user_code_blocks.append("".join(current_block).strip())
  elif current_speaker == "assistant":
    assistant_code_blocks.append("".join(current_block).strip())

  return user_code_blocks, assistant_code_blocks

In [ ]:
#@title Prune Text

import re

code_pattern = re.compile(
    r"```.*?```",
    flags=re.DOTALL
)

title_pattern = re.compile(
    r'^\s*#{3,}.*$',
    re.MULTILINE
)

def prune_code(text):
    cleaned_text = code_pattern.sub('', text)
    return cleaned_text

def prune_titles(text):
    cleaned_text = title_pattern.sub('', text)
    return cleaned_text

In [10]:
#@title Populate Excel Sheet with Conversations

import pandas as pd

file_path = "/content/drive/MyDrive/AI PlugIn Research/GPT 4.0 Data/Multiple_Responses_GPT_Conversations.xlsx"
res_id = 1
for conv_id in range(1,10):
  df = pd.read_excel(file_path)

  text_file_path = f"/content/drive/MyDrive/AI PlugIn Research/GPT 4.0 Data/Multiple Prompt Data/code_dialogue_{conv_id}.txt"
  user_code, assistant_code = extract_code_blocks(text_file_path)

  print("\n\n~~~~Pruning Text~~~~")
  for i in range(len(assistant_code)):
    print(f"\n\n\033[1mAssistant Code: {i + 1}\n\033[0m")
    assistant_code[i] = prune_code(assistant_code[i])
    assistant_code[i] = prune_titles(assistant_code[i])
    print(assistant_code[i])

  new_data = []
  for user, assistant in zip(user_code, assistant_code):
    new_data.append({'conv_id': conv_id, 'res_id': res_id, 'question_text': user, 'answer_text': assistant})
    res_id += 1

  new_df = pd.DataFrame(new_data)
  updated_df = pd.concat([df, new_df], ignore_index=True)
  updated_df.to_excel(file_path, index=False)



~~~~Pruning Text~~~~


Assistant Code: 1

Concurrency issues in a Java application, especially in network-based games like a chess server and client, can be challenging. These issues typically arise when multiple threads try to access shared resources simultaneously, leading to race conditions, deadlocks, or inconsistent data states. Here are some general strategies and specific approaches you might consider to resolve these issues:

   - **Monitor Locks (synchronized keyword)**: Use `synchronized` blocks or methods to ensure that only one thread can execute a block of code at a time.
   
   - **Lock Objects (ReentrantLock)**: More flexible than synchronized blocks, allowing you to try to acquire the lock without blocking indefinitely.

   - Use thread-safe variants of collections from `java.util.concurrent` package, such as `ConcurrentHashMap`, `CopyOnWriteArrayList`, etc., to manage shared data.

   - Use atomic variables from `java.util.concurrent.atomic` package for counters, fla

In [ ]:
#@title Populate Excel Sheet Formatting

if input("Do you want to install xlsxwriter?\n") == "y":
  !pip install xlsxwriter

import openpyxl
import xlsxwriter
import re
import os

# File path for the existing Excel file (we will overwrite this file)
file_path = "/content/drive/MyDrive/AI PlugIn Research/GPT 4.0 Data/Multiple_Responses_GPT_Conversations.xlsx"

# Temporary file path for the new Excel file
temp_file_path = "/content/drive/MyDrive/AI PlugIn Research/GPT 4.0 Data/temp_Multiple_Responses_GPT_Conversations.xlsx"

# Load the existing workbook and select the active sheet
wb_existing = openpyxl.load_workbook(file_path)
ws_existing = wb_existing.active

# Create a new temporary Excel file and add a worksheet
workbook = xlsxwriter.Workbook(temp_file_path)
worksheet = workbook.add_worksheet()

# Define the bold format
bold = workbook.add_format({'bold': True})

# Function to apply bold formatting to text enclosed in **
def apply_bold_xlsxwriter(text, bold_format):
    parts = re.split(r'(\*\*.*?\*\*)', text)
    formatted_text = []
    for part in parts:
        if part.startswith('**') and part.endswith('**'):
            # Remove ** and apply bold
            formatted_text.append(bold_format)
            formatted_text.append(part[2:-2])
        else:
            formatted_text.append(part)
    return formatted_text

# Write headers (assuming the headers are in the first row)
for col_num, cell in enumerate(ws_existing[1], start=0):
    worksheet.write(0, col_num, cell.value)

# Iterate through each cell in the sheet, starting from the second row
for r in range(2, ws_existing.max_row + 1):
    for c in range(1, ws_existing.max_column + 1):
        cell_value = ws_existing.cell(r, c).value
        if cell_value and '**' in str(cell_value):
            # Apply bold formatting to the question and answer text
            formatted_text = apply_bold_xlsxwriter(cell_value, bold)
            worksheet.write_rich_string(r - 1, c - 1, *formatted_text)
        else:
            # Write the value as it is if no formatting is needed
            worksheet.write(r - 1, c - 1, cell_value)

# Close the new workbook
workbook.close()

# Replace the original file with the new formatted file
os.replace(temp_file_path, file_path)

Do you want to install xlsxwriter?
n


/usr/local/lib/python3.10/dist-packages/xlsxwriter/worksheet.py:1344: UserWarning: You must specify more than 2 format/fragments for rich strings. Ignoring input in write_rich_string().
  warn(
